### First open the front end of the Comms channel, in JavaScript

In [20]:
%%js
Jupyter.notebook.kernel.comm_manager.register_target('my_comm_target',
    function(comm, msg) {
        // comm is the frontend comm instance, msg is the comm_open message, which can carry data
        // We don't do anything with the above opening msg
        // Next, we register a handler to deal with later messages.
        // Basically, on receiving a message, we start a timer that posts a send command,
        // which sends the data in a message to the Python back end
        comm.on_msg(function(msg) {
            console.log("Comm message received " + msg.content.data.foo);
            // this function is called by the timer, sends data to the back end
            function myTimer() {      
                const d = new Date();
                comm.send({'time': d.toLocaleString(), 
                           'temp':tempSensor.temperature, 
                           'humidity':humSensor.humidity,
                           'ratio':vrSensor.sensorValue,
                           'lux':liteSensor.illuminance});
            } 
            let myVar = setInterval(myTimer, 300000); // run every 5 minutes (300sec)
        });
        comm.on_close(function(msg) {return 0;});
    });

<IPython.core.display.Javascript object>

### Next, open the back end of the Comms channel, in Python

In [21]:
from ipykernel.comm import Comm

# a global variable we use to keep the data from the front end comm channel
comm_data = "Not set yet"  

# Connect to the comm channel in the front end, to the Python
my_comm = Comm(target_name='my_comm_target', data={'foo': 1})

# Add a callback for received messages. This will call post_stuff to store the data
@my_comm.on_msg
def _recv(msg):
    global comm_data
    comm_data = msg['content']['data']  # this is the data in the comm message
    post_stuff(comm_data)